# Imports

In [107]:
import requests
import bs4
import json
import datetime as dt
import sys
import pandas as pd
import geopy
import warnings
import math
warnings.filterwarnings('ignore')

# Lisbon Properties for Sale - SUPERCASA Webscrapping

In [5]:
url = 'https://supercasa.pt/comprar-casas/lisboa/pagina-1'
result = requests.get(url)
soup = bs4.BeautifulSoup(result.text, 'lxml')
num_of_properties = int(soup.find_all('h1', id='searchTitle')[0].get_text().split()[0].replace('.',''))
num_prop_per_page = 25
total_pages = int(num_of_properties / num_prop_per_page)
time_estimation_for_12505 = 20
time_estimation = round(num_of_properties * 20 / 12505)

print(f'Time estimation for this task: {time_estimation} minutes.')
estimation = (dt.datetime.now() + dt.timedelta(minutes=time_estimation))
start = f'Tasks started at {dt.datetime.now().hour}h:{dt.datetime.now().minute}min. Estimated finish time {estimation.hour}h:{estimation.minute}min.'
print(start)

title, price, num_rooms, total_area, latitude, longitude, region, extras, id = ([] for i in range(9))

for n in range(1, 75):
    url = f'https://supercasa.pt/comprar-casas/lisboa/pagina-{n}'
    result = requests.get(url)
    soup = bs4.BeautifulSoup(result.text, 'lxml')

    # Find all properties on the current page
    properties = soup.find_all('div', class_='property big-picture') 

    for prop in properties:
        # Title
        a = prop.find('h2', class_='property-list-title').find('a')
        title.append(a.get_text().strip() if a else ' ')

        # Price
        span = prop.find('div', class_='property-price').find('span')
        price.append(span.get_text(strip=True) if span else ' ')

        # Features
        feature = prop.find('div', class_='property-features')
        spans = feature.find_all('span') if feature else []
        rooms = spans[0].get_text() if len(spans) > 0 else "Unknown"
        area = spans[1].get_text() if len(spans) > 1 else "Unknown"
        num_rooms.append(rooms)
        total_area.append(area)

        # Links for latitude and longitude
        link = prop.find('a', class_='property-link')
        latitude.append(link.get('data-latitude') if link else 'Unknown')
        longitude.append(link.get('data-longitude') if link else 'Unknown')

        # Extras
        highlight = prop.find('div', class_='property-highlights')
        if highlight:
            extra_spans = highlight.find_all('span')
            extras.append(', '.join([span.get_text(strip=True) for span in extra_spans]))
        else:
            extras.append(' ')

        # Address region from JSON-LD script if necessary
        script = prop.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            if data.get('@type') == 'Offer':
                available_at_or_from = data.get('availableAtOrFrom', {})
                address_info = available_at_or_from.get('address', {})
                address_region = address_info.get('addressRegion', 'Not provided')
                region.append(address_region)
        else:
            region.append('Not provided')

        sys.stdout.write(f"\rProgress: {int((n / total_pages) * 100)}%")
        sys.stdout.flush()

for i in range(len(title)):
    id.append(i)

sys.stdout.write(f"\rProgress: 100%")
sys.stdout.flush()
print('\nCompleted!')

headers = ['id', 'title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'region', 'extras']
final_data = [id, title, price, num_rooms, total_area, latitude, longitude, region, extras]
Lisbon_Properties = pd.DataFrame(dict(zip(headers, final_data)))

print(f"\nYou now have data on {len(Lisbon_Properties['id'])} properties located in Lisbon!")

Time estimation for this task: 20 minutes.
Tasks started at 19h:31min. Estimated finish time 19h:51min.
Progress: 100%
Completed!

You now have data on 1376 properties located in Lisbon!


In [6]:
# Lisbon_Properties.to_csv('Lisbon_Properties.csv')

In [2]:
Lisbon_Properties = pd.read_csv('Lisbon_Properties.csv', index_col=0)
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem


In [3]:
Lisbon_Properties = Lisbon_Properties[~Lisbon_Properties.drop('id', axis=1).duplicated()]

In [4]:
Lisbon_Properties['num_rooms'].value_counts()

num_rooms
2 quartos     426
1 quarto      311
3 quartos     264
4 quartos     136
5 quartos      29
9 quartos      13
7 quartos       6
6 quartos       6
10 quartos      2
8 quartos       1
11 quartos      1
Name: count, dtype: int64

In [5]:
Lisbon_Properties['total_area'].unique()

array(['Área bruta 97 m²', 'Área bruta 90 m²', 'Área bruta 120 m²',
       'Área útil 233 m²', 'Área bruta 93 m²', 'Área bruta 416 m²',
       'Área bruta 55 m²', 'Área bruta 234 m²', 'Área bruta 290 m²',
       'Área bruta 262 m²', 'Área bruta 336 m²', 'Área bruta 264 m²',
       'Área bruta 60 m²', 'Área bruta 874 m²', 'Área bruta 68 m²',
       'Área útil 88 m²', 'Área bruta 48 m²', 'Área bruta 110 m²',
       'Área bruta 102 m²', 'Área útil 142 m²', 'Área bruta 76 m²',
       'Área bruta 119 m²', 'Área bruta 160 m²', 'Área bruta 54 m²',
       'Área bruta 302 m²', 'Área bruta 44 m²', 'Área bruta 150 m²',
       'Área bruta 168 m²', 'Área bruta 70 m²', 'Área bruta 49 m²',
       'Área bruta 230 m²', 'Área bruta 191 m²', 'Área bruta 426 m²',
       'Área bruta 138 m²', 'Área útil 139 m²', 'Área bruta 198 m²',
       'Área bruta 85 m²', 'Área bruta 130 m²', 'Área bruta 180 m²',
       'Área bruta 67 m²', 'Área bruta 134 m²', 'Área bruta 286 m²',
       'Área bruta 77 m²', 'Área bruta 

# Lisbon Metro Info - Wikipedia Webscrapping

In [115]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_esta%C3%A7%C3%B5es_do_Metropolitano_de_Lisboa'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
data = str(soup.find('table', {'class': 'wikitable'}))

table = pd.read_html(data)[0]
columns = ['Nome','Outros nomes','Linha','Lat.','Long.']
Lisbon_Metro = table[columns]

Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003


In [116]:
Lisbon_Metro = Lisbon_Metro[~Lisbon_Metro['Lat.'].astype(str).str.contains('Lat.')]
Lisbon_Metro = Lisbon_Metro[~Lisbon_Metro['Long.'].astype(str).str.contains('Long.')]


In [117]:
Lisbon_Metro['Lat.']

0     38.76861
1     38.73713
2     38.73697
3     38.76038
4     38.74994
5     38.75311
6     38.75847
7     38.77937
8     38.72715
9     38.74233
10    38.73266
11    38.71948
12    38.71051
13    38.71055
14    38.74770
15    38.76293
16    38.70609
17    38.76012
18    38.76027
19    38.74076
20    38.75919
21    38.75479
22    38.75156
23    38.75336
24    38.77502
25    38.74708
26    38.72320
27    38.74140
28    38.74847
29    38.77339
30    38.72431
31    38.72488
32    38.71753
33    38.77488
34    38.79334
35    38.74003
36    38.76101
37    38.76787
38    38.72892
39    38.73037
40    38.76222
41    38.73775
42    38.76737
43    38.72015
44    38.75227
45    38.71590
46    38.74784
47    38.71402
48    38.73471
49    38.73519
50    38.71369
51    38.73392
52    38.73476
53    38.78564
54    38.75997
55    38.70703
Name: Lat., dtype: object

In [118]:
Lisbon_Metro['NomeConcat'] = Lisbon_Metro['Nome']+' - '+Lisbon_Metro['Outros nomes']
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.,NomeConcat
0,Aeroporto,—,Vermelha,38.76861,−9.12861,Aeroporto - —
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388,Alameda - Alameda I (técn.)
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261,Alameda - Alameda II (técn.)
3,Alfornelos,—,Azul,38.76038,−9.20435,Alfornelos - —
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003,Alto dos Moinhos - Centro Administrativo (prev.)


# Data

In [8]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem


In [9]:
def extra_rooms(x):
    if '+' in x:
        return int(x.split('+')[1])
    else:
        return 0

def area(x):
    if 'Área' in x:
        return int(x.split()[2].replace('.',''))
    else:
        return 'Unknown'

def num_extras(x):
    if x.isspace():
        return 0
    elif ',' in x:
        return int(len(x.split(',')))
    else:
        return 1

In [119]:
#Properties Transformation
Lisbon_Properties['Type'] = Lisbon_Properties['title'].apply(lambda x: x.split()[0])
Lisbon_Properties['Typology'] = Lisbon_Properties['title'].apply(lambda x: x.split()[1])
Lisbon_Properties['Extra_Rooms'] = Lisbon_Properties['Typology'].apply(extra_rooms)
Lisbon_Properties['Extra_Rooms_Flag'] = Lisbon_Properties['Extra_Rooms'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['N_Rooms'] = Lisbon_Properties['num_rooms'].apply(lambda x: int(x.split()[0]))
Lisbon_Properties['Total_N_Rooms'] = Lisbon_Properties['N_Rooms'] + Lisbon_Properties['Extra_Rooms']
Lisbon_Properties['Price'] = Lisbon_Properties['price'].apply(lambda x: int(x.replace('.','').split()[0]))
Lisbon_Properties['Area_m2'] = Lisbon_Properties['total_area'].apply(area)
Lisbon_Properties['N_Extras'] = Lisbon_Properties['extras'].apply(num_extras)
Lisbon_Properties['Extras_Flag'] = Lisbon_Properties['N_Extras'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['Latitude'] = Lisbon_Properties['latitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties['Longitude'] = Lisbon_Properties['longitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties.rename(columns={'region':'Region'}, inplace=True)

#Metro Transformation
Lisbon_Metro['Lat.'] = Lisbon_Metro['Lat.'].str.replace('−', '-').astype(float)
Lisbon_Metro['Long.'] = Lisbon_Metro['Long.'].str.replace('−', '-').astype(float)



In [86]:
Lisbon_Metro.dtypes


Nome            object
Outros nomes    object
Linha           object
Lat.            object
Long.           object
dtype: object

In [11]:
extras_df = Lisbon_Properties['extras'].str.split(',', expand=True)

extras = []
unique_extras = []

for x in [0,1,2,3,4]:
    extras.append(list(extras_df[x].unique()))

for ext in extras:
    unique_extras += ext

unique_extras = set(unique_extras)
unique_extras = {extra.strip() for extra in unique_extras if extra is not None and extra.strip()}

unique_extras


{'Com elevador',
 'Com garagem',
 'De luxo',
 'Piscina',
 'Rés do chão',
 'Vista para mar',
 'Último andar'}

In [12]:
for feature in unique_extras:
    Lisbon_Properties[feature] = Lisbon_Properties['extras'].apply(lambda x: int(feature in x))

In [13]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,Region,extras,Type,...,Extras_Flag,Latitude,Longitude,Com garagem,Rés do chão,De luxo,Último andar,Com elevador,Vista para mar,Piscina
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,,Apartamento,...,0,38.751710,-9.20090,0,0,0,0,0,0,0
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,,Apartamento,...,0,38.745739,-9.14259,0,0,0,0,0,0,0
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,,Apartamento,...,0,38.746570,-9.17989,0,0,0,0,0,0,0
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem",Apartamento,...,1,38.707940,-9.14468,1,0,0,0,1,0,0
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem,Apartamento,...,1,38.710780,-9.14385,1,0,0,0,0,0,0


In [14]:
region_dummies = pd.get_dummies(Lisbon_Properties['Region'],dtype=int)
type_dummies = pd.get_dummies(Lisbon_Properties['Type'],dtype=int)

In [15]:
Lisbon_Properties = pd.concat([Lisbon_Properties, region_dummies, type_dummies], axis=1)

In [16]:
Lisbon_Properties.drop(['Region','Type'], axis=1, inplace=True)

In [17]:
Lisbon_Properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1195 entries, 0 to 1380
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1195 non-null   int64  
 1   title                    1195 non-null   object 
 2   price                    1195 non-null   object 
 3   num_rooms                1195 non-null   object 
 4   total_area               1195 non-null   object 
 5   latitude                 1195 non-null   object 
 6   longitude                1195 non-null   object 
 7   extras                   1195 non-null   object 
 8   Typology                 1195 non-null   object 
 9   Extra_Rooms              1195 non-null   int64  
 10  Extra_Rooms_Flag         1195 non-null   int64  
 11  N_Rooms                  1195 non-null   int64  
 12  Total_N_Rooms            1195 non-null   int64  
 13  Price                    1195 non-null   int64  
 14  Area_m2                  1195

In [18]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,extras,Typology,Extra_Rooms,...,São Vicente,Apartamento,Casa,Duplex,Flat,Loft,Moradia,Palacete,Penthouse,Quinta
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",,T1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",,T2,0,...,0,1,0,0,0,0,0,0,0,0
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",,T3,0,...,0,1,0,0,0,0,0,0,0,0
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468","Com elevador, Com garagem",T4+1,1,...,0,1,0,0,0,0,0,0,0,0
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Com garagem,T1,0,...,0,1,0,0,0,0,0,0,0,0


##On stations bring the nb of stations near a house instead of flagging which stations are in the vicinities. 
Alternative is to flag whether or not there's a metro nearby.

In [129]:
# Haversine function to calculate the distance between two points on the Earth
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radius of Earth in kilometers
    R = 6371.0
    
    # Calculate the distance
    distance = R * c
    
    return distance

#Define Distance Threshold
dist_threshold = 1.5

# Prepare a list to store the distances
distances = []

# Calculate distances between each property and each metro station
for p_index, p_row in Lisbon_Properties.iterrows():
    count_nearby_stations = 0
    for m_index, m_row in Lisbon_Metro.iterrows():
        distance = haversine(p_row['Latitude'], p_row['Longitude'], m_row['Lat.'], m_row['Long.'])
        if distance < dist_threshold:
            count_nearby_stations += 1
        distances.append({
            'property_id': p_row['id'],
            'property_title': p_row['title'],
            'metro_station': m_row['Nome'],
            'distance_km': distance
        })
    Lisbon_Properties.at[p_index, 'stations_within_1.5km'] = count_nearby_stations

# Create a DataFrame from the distances list
Distances_df = pd.DataFrame(distances)

# Print the DataFrame with distances
Distances_df.head()



,property_id,property_title,metro_station,distance_km
0,0,"Apartamento T1 em Benfica, Lisboa",Aeroporto,6.543674
1,0,"Apartamento T1 em Benfica, Lisboa",Alameda,6.034241
2,0,"Apartamento T1 em Benfica, Lisboa",Alameda,6.145132
3,0,"Apartamento T1 em Benfica, Lisboa",Alfornelos,1.009409
4,0,"Apartamento T1 em Benfica, Lisboa",Alto dos Moinhos,1.820479


In [127]:
Distances_df.loc[Distances_df['property_id'] == 0]

,property_id,property_title,metro_station,distance_km
0,0,"Apartamento T1 em Benfica, Lisboa",Aeroporto,6.543674
1,0,"Apartamento T1 em Benfica, Lisboa",Alameda,6.034241
2,0,"Apartamento T1 em Benfica, Lisboa",Alameda,6.145132
3,0,"Apartamento T1 em Benfica, Lisboa",Alfornelos,1.009409
4,0,"Apartamento T1 em Benfica, Lisboa",Alto dos Moinhos,1.820479
5,0,"Apartamento T1 em Benfica, Lisboa",Alvalade,4.940078
6,0,"Apartamento T1 em Benfica, Lisboa",Amadora Este,1.664757
7,0,"Apartamento T1 em Benfica, Lisboa",Ameixoeira,4.718982
8,0,"Apartamento T1 em Benfica, Lisboa",Anjos,6.346299
9,0,"Apartamento T1 em Benfica, Lisboa",Areeiro,5.934031


In [128]:
# Print the updated Lisbon_Properties DataFrame with the new column
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,extras,Typology,Extra_Rooms,...,Casa,Duplex,Flat,Loft,Moradia,Palacete,Penthouse,Quinta,Type,stations_within_1.5km
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",,T1,0,...,0,0,0,0,0,0,0,0,Apartamento,4.0
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",,T2,0,...,0,0,0,0,0,0,0,0,Apartamento,9.0
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",,T3,0,...,0,0,0,0,0,0,0,0,Apartamento,4.0
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468","Com elevador, Com garagem",T4+1,1,...,0,0,0,0,0,0,0,0,Apartamento,8.0
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Com garagem,T1,0,...,0,0,0,0,0,0,0,0,Apartamento,9.0


<html>
<p><strong>Great Circle Distance Formula:</strong></p>
<p>The formula to calculate the great circle or 'as the crow flies' distance between two points on the Earth's surface, given their latitude and longitude is:</p>
<p style="font-family: 'Lucida Console', Monaco, monospace;">
  \( d = 2R \times \sin^{-1}\left(\sqrt{\sin^2\left(\frac{\theta_2 - \theta_1}{2}\right) + \cos \theta_1 \times \cos \theta_2 \times \sin^2\left(\frac{\phi_2 - \phi_1}{2}\right)}\right) \)
</p>
<p>where:</p>
<ul>
  <li><strong>\( (\theta_1, \phi_1) \)</strong> and <strong>\( (\theta_2, \phi_2) \)</strong> – Coordinates of each point (latitude and longitude, respectively);</li>
  <li><strong>R</strong> – Radius of the Earth; and</li>
  <li><strong>d</strong> – Great circle distance between the points.</li>
</l>
</html>


In [ ]:
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003


##Keep this cell LAST 

In [ ]:
columns_to_drop = ['title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'Typology','extras']
Lisbon_Properties = Lisbon_Properties.drop(columns_to_drop, axis=1)